In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.fundaments.thing import Thing

In [ ]:
from everest.ptolemaic.fundaments.intt import Intt

In [ ]:
Intt.Brace[1, 2]

In [ ]:
1 in Intt

In [ ]:
Intt(1)

In [ ]:
Intt[10:]

In [ ]:
Intt.Brace.__mro__

In [ ]:
Intt.Brace.SubmemberType

In [ ]:
Intt.Brace.Oid.SubmemberType

In [ ]:
Intt.Oid.SubmemberType

In [ ]:
Intt.Brace((1, 2))

In [ ]:
Intt.Brace[1, 2]

In [ ]:
1 in _Brace

In [ ]:
_Brace.__incision_manager__

In [ ]:
(1, 2) in _Brace

In [ ]:
Cell((1, 2)) in Cell

In [ ]:
Cell((1, 2))

In [ ]:
Cell[1, 2]

In [ ]:
Cell[2]